In [1]:

import torch
import argparse
from pathlib import Path
import numpy as np
from src.model_utils import build_model
import src.datasets as datasets
import src.evaluation as evaluation
from src.torch_utils import torch2numpy
from reevaluate import get_test_dataset
from tqdm import tqdm


/usr/local/Caskroom/miniforge/base/envs/py3/lib/python3.7/site-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [2]:
def get_quantile_forecast(pred, quantile):
    # shape is [num_samples, num_series, prediction_length]
    # return the quantile of the samples for each series
    return np.quantile(pred, quantile, axis=0)


In [26]:
ckpt_file = "./Checkpoints/bouncing_ball.pt"
ckpt = torch.load(ckpt_file, map_location="cpu")

config = ckpt["config"]
model = build_model(config=config)
model.load_state_dict(ckpt["model"])

test_dataset = get_test_dataset(config)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=80)
extra_args = dict()
extra_args = {"dur_temperature": 1.0}
device = 'cpu'

In [4]:
def inference(ckpt_file, dataset_path):

    ckpt = torch.load(ckpt_file, map_location="cpu")

    config = ckpt["config"]
    model = build_model(config=config)
    model.load_state_dict(ckpt["model"])

    #test_dataset = get_test_dataset(config)
    test_dataset = datasets.BouncingBallDataset(path=dataset_path)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=80)

    #do predictions autoregressively. 
    context_length = config['context_length']
    prediction_length = config['prediction_length']
    ground_truth = []
    all_mean = []
    # calculate coverage and width of prediction intervals
    for test_batch, test_label in test_loader:
        true = test_batch[:, -prediction_length:]
        autoregressive_mean = []
        for t in tqdm(range(prediction_length)):
            pred = model.predict(test_batch[:,t:t+context_length], num_samples=1, pred_one_step=True)
            pred_y = pred['rec_n_forecast'][:,:,-1]
            autoregressive_mean.append(pred_y)
        ground_truth.append(true)
        all_mean.append(np.concatenate(autoregressive_mean, -1).squeeze())

    ground_truth = np.concatenate(ground_truth, 0)
    all_mean = np.concatenate(all_mean, 0)

    return ground_truth, all_mean


In [40]:
ckpt = torch.load(ckpt_file, map_location="cpu")

dataset_path = "./data/bouncing_ball_calibration.npz"
config = ckpt["config"]
model = build_model(config=config)
model.load_state_dict(ckpt["model"])

#test_dataset = get_test_dataset(config)
test_dataset = datasets.BouncingBallDataset(path=dataset_path)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=80)

#do predictions autoregressively. 
context_length = config['context_length']
prediction_length = config['prediction_length']
ground_truth = []
all_mean = []
# calculate coverage and width of prediction intervals
for test_batch, test_label in test_loader:
    true = test_batch[:, -prediction_length:]
    autoregressive_mean = []
    for t in tqdm(range(prediction_length)):
        pred = model.predict(test_batch[:,t:t+context_length], num_samples=1, pred_one_step=True)
        pred_y = pred['rec_n_forecast'][:,:,-1]
        autoregressive_mean.append(pred_y)
    ground_truth.append(true.squeeze())
    all_mean.append(np.concatenate(autoregressive_mean, -1).squeeze())
    break

  0%|          | 0/50 [00:00<?, ?it/s]/Users/sophiasun/Desktop/2cool4school/phd/switching/REDSDS/src/torch_utils.py:37: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices = indices // dim
100%|██████████| 50/50 [00:05<00:00,  8.82it/s]


In [41]:
ground_truth[0].shape

torch.Size([80, 50])

In [44]:
all_mean[0].shape


(80, 50)

In [39]:
np.concatenate(autoregressive_mean, -1).squeeze().shape

(80, 50)

In [5]:
ckpt_bb = "./Checkpoints/bouncing_ball.pt"
ckpt_bbnoisy = "./Checkpoints/bb_noisy.pt"


In [6]:
ground_truth, all_mean = inference(ckpt_bb, "./data/bouncing_ball_calibration.npz")
np.savez("./results/bouncing_ball_calibration.npz", ground_truth=ground_truth, mean=all_mean)


  0%|          | 0/50 [00:00<?, ?it/s]/Users/sophiasun/Desktop/2cool4school/phd/switching/REDSDS/src/torch_utils.py:37: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices = indices // dim
100%|██████████| 50/50 [00:03<00:00, 12.50it/s]
<string>:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.

In [7]:
ground_truth, all_mean = inference(ckpt_bbnoisy, "./data/bouncing_ball_noisy_calibration.npz")
np.savez("./results/bouncing_ball_noisy_calibration.npz", ground_truth=ground_truth, mean=all_mean)


100%|██████████| 50/50 [00:03<00:00, 13.64it/s]


In [18]:
#calculate errors on the calibration dataset

preds = np.load("./results/bouncing_ball_calibration.npz", allow_pickle=True)
ground_truth = preds["ground_truth"]
mean = preds["mean"]


In [19]:
pred_mean = np.concatenate(mean, 1)

In [24]:
mean[0].shape

torch.Size([1, 80, 1])

In [22]:
ground_truth.shape

(3000, 50, 1)

In [ ]:

mses = (ground_truth - mean) ** 2


## Forcasting

In [ ]:

pred_segs = []
true_segs = []
for test_batch, test_label in tqdm(test_loader):
    test_batch = test_batch.to(device)
    pred = model.predict(test_batch, num_samples=100)
    true = test_batch[:, config['context_length']:]
    pred_segs.append(torch2numpy(torch.argmax(pred['z_emp_probs'], dim=-1)))
    true_segs.append(torch2numpy(test_label[:, config['context_length']:]))


In [106]:
seg_error = evaluation.evaluate_segmentation(np.concatenate(true_segs, 0), np.concatenate(pred_segs, 0), K=config["num_categories"])

seg_error

{'nmi_score': 0.10380844559277072,
 'ari_score': 0.140397887557092,
 'accuracy': 0.68736,
 'f1_score': 0.6873676161666078}

In [84]:
np.mean(mses)

0.08520873

In [97]:
pred_segs

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])

## Calibration and conformal prediction